In [17]:
# combine charts.csv and hot100.csv
import pandas as pd
from termcolor import colored

path_hot100 = '../data/ranking_5/hot100.csv'
path_charts = '../data/ranking_5/charts.csv'

# read&load data
try:
    df_hot = pd.read_csv(path_hot100, dtype={4: str})
    df_charts = pd.read_csv(path_charts)
    print(colored(f"Success！\nHot100 shape: {df_hot.shape}\nCharts shape: {df_charts.shape}", 'green'))
except FileNotFoundError:
    print(colored("Wrong!", 'red'))


Success！
Hot100 shape: (350787, 8)
Charts shape: (330087, 7)


In [18]:
# unify column name
df_hot = df_hot.drop(columns=['Image URL'])
df_hot.columns = ['date', 'song', 'artist', 'rank', 'last_week', 'peak_rank', 'weeks_on_board']
df_charts.columns = ['date', 'rank', 'song', 'artist', 'last_week', 'peak_rank', 'weeks_on_board']

In [ ]:
# modify data type


In [19]:
# overall comparison
df_hot.info()
df_hot.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350787 entries, 0 to 350786
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   date            350787 non-null  object
 1   song            350787 non-null  object
 2   artist          350787 non-null  object
 3   rank            350787 non-null  int64 
 4   last_week       350787 non-null  object
 5   peak_rank       350787 non-null  int64 
 6   weeks_on_board  350787 non-null  object
dtypes: int64(2), object(5)
memory usage: 18.7+ MB


,rank,peak_rank
count,350787.000000,350787.000000
mean,50.500882,32.836074
std,28.866097,27.718423
min,1.000000,1.000000
25%,26.000000,6.000000
50%,51.000000,27.000000
75%,76.000000,55.000000
max,100.000000,100.000000


In [20]:
df_charts.info()
df_charts.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330087 entries, 0 to 330086
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   date            330087 non-null  object 
 1   rank            330087 non-null  int64  
 2   song            330087 non-null  object 
 3   artist          330087 non-null  object 
 4   last_week       297775 non-null  float64
 5   peak_rank       330087 non-null  int64  
 6   weeks_on_board  330087 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 17.6+ MB


,rank,last_week,peak_rank,weeks_on_board
count,330087.000000,297775.000000,330087.000000,330087.000000
mean,50.500929,47.591631,40.970629,9.161785
std,28.866094,28.054360,29.347481,7.618264
min,1.000000,1.000000,1.000000,1.000000
25%,26.000000,23.000000,13.000000,4.000000
50%,51.000000,47.000000,38.000000,7.000000
75%,76.000000,72.000000,65.000000,13.000000
max,100.000000,100.000000,100.000000,90.000000


In [ ]:
#unify data type